In [1]:
"""Use of shanghai_platform
one simulation, several draws"""




#TODO

#put in the module
#create data collection
#create new file for automation

'Use of shanghai_platform\none simulation, several draws'

In [2]:
# 0.1 Modules

from CircleNet.shape import circle,Timer
from CircleNet.shanghai_platform import Simulation, MatchingPlatform, Driver, Passenger
from  CircleNet.shanghai_platform_extension import Positions_drawing

In [3]:
# 0.2 Help

#help(MatchingPlatform)
#print(Driver.attributes), print(Passenger.attributes)

In [4]:
# 1.1 Parameters

speed=25 #km/h
R=25 #km
end=3 #h

N_driver=600
N_passenger=600


#in second :
#Drivers:
watching_repetition_time=60
first_watching_before_first_departure=5*60
window_size_of_departure=15*60
time_elasticity=5*60
fuel_cost=0.5
time_perception=5

#Passenger
publishing_advance=20*60

In [5]:
# 1.2 Simulation type

#transformation in m, s
speed/=3.6
R*=1000
end*=3600

def incentive(b):
    def o(*a,**k):
        return b(*a,**k)+500
    return o

#benefit for a drive
@incentive
def benefits(origin,destination,network):
    """Shanghai pricing"""
    distance=network.travel_distance(origin,destination)
    if distance < 3000: #3n first km fixprice
        return 11#in RMB
    if distance < 20000:#until 20km at 1.5RMB / km
        return 6.5 + distance * 1.5 # 11 + (distance - 3) * 1.5
    return 8.5 + distance # 28.5 + (distance - 20) *1   # 28.5 = 3 + (20 - 3) * 1.5



#agents generator
def SimpleDriver(simulation):
    t=simulation.timer.random_time()
    O=simulation.network.position_generator()
    D=simulation.network.position_generator()
    w=(t+first_watching_before_first_departure,t+first_watching_before_first_departure+window_size_of_departure)
    A=w[1]+simulation.network.travel_time(O,D)+time_elasticity
    return Driver(first_watching_time=t,
                  repetition_time=watching_repetition_time,
                  departure_window=w,
                  position=O,destination=D,
                  last_arrival_time=A,
                  fuel_cost=fuel_cost,
                  time_perception=time_perception)
def SimplePassenger(simulation):
    t=simulation.timer.random_time()
    O=simulation.network.position_generator()
    D=simulation.network.position_generator()
    return Passenger(publishing_time=t,
                     last_departure_time=t+publishing_advance,
                     position=O,destination=D)


In [6]:
#1.3 Simulation construction

N=circle(R,speed)
T=Timer(end)
simu=Simulation(N,T)


simu.matchingAlgo=MatchingPlatform(benefits,simu)


for i in range(N_driver):
    simu.add(SimpleDriver(simu))
for i in range(N_passenger):
    simu.add(SimplePassenger(simu))
    



In [7]:
#2. Execution

#simu.set_action(print,)
simu()

In [8]:
#3.1 Observations

nb_driver_alone=0
for p in simu:
    if isinstance(p,Driver):
        for t,action in p.story:
            if action[0] is "alone":
                nb_driver_alone+=1
                break
print(nb_driver_alone,"/",N_driver)

nb_passenger_missed=0
for p in simu:
    if isinstance(p,Passenger):
        for action in p.story.actions:
            if action[0] is "missed":
                nb_passenger_missed+=1
                break
print(nb_passenger_missed,"/",N_passenger)

415 / 600
415 / 600


In [9]:
# 3.2 Animation

dr=Positions_drawing(simu)
dr.set_options(margin=20,real_size=(10,10),time_interval=30,time_coef=1000,repetition=True,hide_axes=True)
#%matplotlib notebook
dr()

In [10]:
# 3.3 Route representation
import matplotlib.pyplot as plt
from math import sqrt
import matplotlib.lines as mlines


#data extraction
def driver_matched_route_extraction(simu):
    points=[]
    for agent in simu:
        if isinstance(agent,Driver):
            interesting=False
            for _,action in agent.story:
                if action[0] is "matched":#we want his route !
                    interesting=True
                    break
            if interesting :
                route=[]
                for _,action in agent.story.reverse_iter():
                    if action[0] is "moving" :
                        route.append(action[1]["start"])
                    elif action[0] is "arrived":
                        route.append(action[1]["point"])
                points.append(route)
    return points
def transpose(M):
    """transform M[i][j][k] in M[k][j][i]"""
    return zip(*[zip(*t) for t in zip(*M)])


    
#color determination
def color_function(m,M,cm,cM):
    """creates a function that attributes a (R,G,B) color for a value between m and M
    m has cm as a color
    M has cM
    the transformtion is linear"""
    def outf(value):
        value=(value-m)/(M-m)#between 0 and 1
        return (cm[0]*(1-value)+cM[0]*value,cm[1]*(1-value)+cM[1]*value,cm[2]*(1-value)+cM[2]*value)
    return outf
def length(trajectories):
    """length of basic travel : from first point to last one"""
    out=[]
    for t in trajectories:
        last=len(t)-1
        size=sqrt((t[0][0]-t[last][0])**2+(t[0][1]-t[last][1])**2)
        out.append(size)
    return out
def color_list(trajectories,cm,cM):
    l=length(trajectories)
    function=color_function(min(l),max(l),cm,cM)
    legend=[mlines.Line2D([], [], color=cm, markersize=15, label=str(round(min(l)))+' m'),
            mlines.Line2D([], [], color=cM, markersize=15, label=str(round(max(l)))+' m')]
    out=[]
    for v in l:
        out.append(function(v))
    return out,legend

#spatial transformations
def translation(points):
    x,y=points[0]
    return [(p[0]-x,p[1]-y) for p in points]
def rescale(points):
    x,y=points[len(points)-1]
    s=sqrt(x*x+y*y)
    return [(p[0]/s,p[1]/s) for p in points]
def turn(points):
    x,y=points[len(points)-1]
    return [(y*p[0]-x*p[1],x*p[0]+y*p[1]) for p in points]
def arrange(points):
    return turn(rescale(translation(points)))





#route plot functions
def plot_routes(simu):
    P=driver_matched_route_extraction(simu)
    plt.plot(*transpose(P))
    plt.axis('equal')
    plt.show()
    
def plot_relative_routes(simu,cm=(1,0.2,0),cM=(0,0.2,1)):
    P=list(driver_matched_route_extraction(simu))
    colors,legend=color_list(P,cm,cM)
    plt.gca().set_prop_cycle(color=colors)
    P=[arrange(r) for r in P]
    plt.plot(*transpose(P))
    plt.axis('equal')
    plt.legend(handles=legend)
    plt.show()

In [11]:
#simple route plot

plot_routes(simu)






In [13]:
#aggregated

plot_relative_routes(simu)

